## Heston model with Euler-Murayama

Solve SDE for Heston model using the Euler-Maruyama with $N_0, 2*N_0,\dots,2^L\times N_0$ time steps. Compute strong and weak rates using overkill solution with $2^{(L+extra)}\times N_0$ time steps and all paths generated at the same time.

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import time
def sde_heston2():
    # solve SDE for Heston model
    # using Euler-Maruyama with N0,2*N0,...,2^L*N0 time steps
    # compute strong and weak rates using overkill solution with 2^(L+extra)*N0 time steps
    # all paths generated at the same time
    
    N0 = 10                            # number of steps on coarsest level
    L = 5                              # number of refinement steps
    M = 10**4                          # number of samples
    T = 1                              # final time
    x0 = 10                            # initial condition for X_t
    v0 = 0.5                           # initial value for volatility
    extra = 3                          # extra levels of refinement for overkill solution
    xi = 0.25
    theta = 0.5
    r = 0.05
    kappa = 2
    G = lambda x: np.maximum(11 - x, 0)
    
    #####################
    tic = time.time()
    Le = L + extra
    Ne = N0*2**Le
    BI = brownp(T, Ne, M)              # path of Brownian motion on finest level
    BII = brownp(T, Ne, M)
    YT = np.zeros((L+2, M))           # values of X_T for h=T/(N*2^l), l=0,...,Le
    Lv = np.arange(L+1)
    Lv = np.concatenate([Lv, [Le]])
    # loop over levels
    for l in range(L+2): # l=0,...,L,Le
        le = Lv[l]
        N = N0*2**le
        p = 2**(Le-le)               # p = Ne/N
        h = T/N
        x = x0
        v = v0
        # loop over increments
        for j in range(N):
            # perform N steps of E-M method
            idx1 = 1 + j*p
            idx2 = 1 + (j-1)*p
            if idx1 >= len(BI) or idx2 >= len(BI):
                break  # terminate the loop if indices are out of bounds
            dBI = BI[idx1, :] - BI[idx2, :]  # Brownian process increment
            dBII = BII[idx1, :] - BII[idx2, :]  # Brownian process increment
            x = x + r*x*h + np.multiply((np.abs(v))**0.5, np.multiply(x,dBI))  # Euler-Maruyama step
            v = v + kappa*(theta-v)*h + xi*(np.abs(v)**0.5)*dBII  # Euler-Maruyama step
        #
        YT[l, :] = x               # values of X_T
    
    Ys = G(YT)                     # compute payoffs
    YTe = np.abs(YT[0:L+1, :] - np.tile(YT[L+1, :], (L+1, 1)))  # errors for XT compared to overkill solution
    YTem = np.mean(YTe, axis=1)   # mean errors for strong convergence
    YTem2 = np.sqrt(np.sum(YTe**2, axis=1))                    # mean errors for strong convergence
    Ym = np.mean(Ys, axis=1)      # sample means for Y
    est_var = np.var(Ys[0:-1, :], ddof=1, axis=1)
    
    AM = Ym[0:-1] - 1.96*np.sqrt(est_var/M)  # based on CLT.
    BM = Ym[:-1] + 1.96*np.sqrt(est_var/M) # based on CLT.
    hv = T/(N0*2**(np.arange(0, L+1))) # vector of h values
    hL = hv[-1]
    p = hv[0]/hL

    print('CLT confidence interval')
    print(np.array([AM, BM]).T)

    # Plots
    fig, ax = plt.subplots()
    ax.loglog(hv, YTem2, '-gx', label='strong error in L^1')
    ax.loglog(hv, YTem, '-ro', label='strong error in L^2')
    ax.loglog(hL*np.array([1, p]), YTem[-1]*np.array([1, p**0.5]), 'k', label='$h^{1/2}$')
    ax.loglog(hL*np.array([1, p]), YTem2[-1]*np.array([1, p**0.5]), 'k--', label='$h^{1/2}$')
    ax.legend()
    ax.grid(True, which='both')
    ax.set_xlabel('step size $h$')
    ax.set_ylabel('error')
    plt.show()


In [11]:
sde_heston2()

NameError: name 'brownp' is not defined